<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/vacuum_stabilization_full_fixed_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.calibration import calibration_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ------------------------------------------------------------------------------
# 1. Model Definition with Dropout for MC‐Dropout
# ------------------------------------------------------------------------------
class VacuumAI(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, p_dropout=0.2):
        super(VacuumAI, self).__init__()
        self.fc1    = nn.Linear(input_dim, hidden_dim)
        self.ln1    = nn.LayerNorm(hidden_dim)
        self.act    = nn.GELU()
        self.drop1  = nn.Dropout(p_dropout)

        self.fc2    = nn.Linear(hidden_dim, hidden_dim // 2)
        self.ln2    = nn.LayerNorm(hidden_dim // 2)
        self.drop2  = nn.Dropout(p_dropout)

        self.cls_fc = nn.Linear(hidden_dim // 2, n_classes)
        self.reg_fc = nn.Linear(hidden_dim // 2, 1)

    def forward(self, x):
        h = self.act(self.ln1(self.fc1(x)))
        h = self.drop1(h)
        h = self.act(self.ln2(self.fc2(h)))
        h = self.drop2(h)

        logits = self.cls_fc(h)
        logp   = F.log_softmax(logits, dim=-1)
        energy = self.reg_fc(h)
        return logp, energy, h

# ------------------------------------------------------------------------------
# 2. Synthetic Dataset (Replace with your real data)
# ------------------------------------------------------------------------------
class SyntheticVacuumDataset(Dataset):
    def __init__(self, N=20000):
        # 4 fluctuation variables
        X = np.random.randn(N, 4).astype(np.float32)

        # Simple toy labels by sum-of-features
        s = X.sum(axis=1)
        y_cls = np.zeros(N, dtype=np.int64)
        y_cls[s > -0.5] = 1
        y_cls[s >  0.5] = 2

        # Toy vacuum-energy: sum of squares
        y_eng = (X**2).sum(axis=1, keepdims=True).astype(np.float32)

        self.X      = torch.from_numpy(X)
        self.y_cls  = torch.from_numpy(y_cls)
        self.y_eng  = torch.from_numpy(y_eng)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y_cls[idx], self.y_eng[idx]

# ------------------------------------------------------------------------------
# 3. Temperature Scaler for Calibration
# ------------------------------------------------------------------------------
class TemperatureScaler(nn.Module):
    def __init__(self):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones(1))

    def forward(self, logit):
        return logit / self.temperature

    def set_temperature(self, valid_loader, model, device):
        nll_criterion = nn.NLLLoss()
        self.to(device)
        model.eval()

        logits_list, labels_list = [], []
        with torch.no_grad():
            for xb, yb, _ in valid_loader:
                xb = xb.to(device)
                logp, _ = model(xb)[:2]
                logits_list.append(logp.cpu())
                labels_list.append(yb)
        logits = torch.cat(logits_list)
        labels = torch.cat(labels_list)

        optimizer = torch.optim.LBFGS([self.temperature], lr=0.1, max_iter=50)

        def _eval():
            optimizer.zero_grad()
            loss = nll_criterion(self.forward(logits), labels)
            loss.backward()
            return loss

        optimizer.step(_eval)
        return self.temperature.item()

# ------------------------------------------------------------------------------
# 4. Physics‐Informed Penalty: enforce energy ≥ 0
# ------------------------------------------------------------------------------
def physics_penalty(energy_pred):
    return torch.mean(F.relu(-energy_pred))

# ------------------------------------------------------------------------------
# 5. Prepare DataLoaders
# ------------------------------------------------------------------------------
dataset     = SyntheticVacuumDataset(N=20000)
train_size  = int(0.8 * len(dataset))
val_size    = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=256)

# ------------------------------------------------------------------------------
# 6. Instantiate Model, Losses, Optimizer, Scheduler, Early Stopping
# ------------------------------------------------------------------------------
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = VacuumAI(input_dim=4, hidden_dim=64, n_classes=3).to(device)

cls_loss  = nn.NLLLoss()
reg_loss  = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

best_val   = float('inf')
patience   = 0
max_patience = 10
history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

# ------------------------------------------------------------------------------
# 7. Training & Validation Loop
# ------------------------------------------------------------------------------
for epoch in range(1, 101):
    # Training
    model.train()
    total_loss = 0.0
    for xb, yb_cls, yb_eng in train_loader:
        xb, yb_cls, yb_eng = xb.to(device), yb_cls.to(device), yb_eng.to(device)
        optimizer.zero_grad()
        logp, eng_pred, _ = model(xb)
        loss = (
            cls_loss(logp, yb_cls)
            + 0.5 * reg_loss(eng_pred, yb_eng)
            + 0.1 * physics_penalty(eng_pred)
        )
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    train_loss = total_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for xb, yb_cls, yb_eng in val_loader:
            xb, yb_cls, yb_eng = xb.to(device), yb_cls.to(device), yb_eng.to(device)
            logp, eng_pred, _ = model(xb)
            loss = (
                cls_loss(logp, yb_cls)
                + 0.5 * reg_loss(eng_pred, yb_eng)
                + 0.1 * physics_penalty(eng_pred)
            )
            val_loss += loss.item() * xb.size(0)
            preds = logp.argmax(dim=1)
            correct += (preds == yb_cls).sum().item()
            total += xb.size(0)
    val_loss /= len(val_loader.dataset)
    val_acc  = correct / total

    # Scheduler & Early Stopping
    scheduler.step(val_loss)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), 'best_vacuumai.pt')
        patience = 0
    else:
        patience += 1
        if patience >= max_patience:
            print(f"Early stopping at epoch {epoch}")
            break

    print(
        f"Epoch {epoch:03d} | Train Loss: {train_loss:.4f} "
        f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.3f}"
    )

# ------------------------------------------------------------------------------
# 8. Load Best Model & Calibrate with Temperature Scaling
# ------------------------------------------------------------------------------
model.load_state_dict(torch.load('best_vacuumai.pt'))
temp_scaler = TemperatureScaler()
T_opt = temp_scaler.set_temperature(val_loader, model, device)
print(f"Optimal temperature: {T_opt:.3f}")

# ------------------------------------------------------------------------------
# 9. MC‐Dropout Uncertainty Estimation (fixed)
# ------------------------------------------------------------------------------
def mc_dropout_predict(x, model, n_samples=50):
    """
    Returns mean and std of predicted class probabilities
    using MC‐Dropout (keeps dropout active but no grad tracking).
    """
    model.train()  # keep dropout on
    probs_stack = []
    with torch.no_grad():
        for _ in range(n_samples):
            logp, _, _ = model(x.to(device))
            probs = torch.exp(logp)
            probs_stack.append(probs.cpu().numpy())
    probs_stack = np.stack(probs_stack, axis=0)
    mean_prob = probs_stack.mean(axis=0)
    std_prob  = probs_stack.std(axis=0)
    return mean_prob, std_prob

# Example on one validation batch
xb, yb_cls, _ = next(iter(val_loader))
mean_p, std_p = mc_dropout_predict(xb, model, n_samples=50)
print("MC‐Dropout mean probabilities:", mean_p[0])
print("MC‐Dropout uncertainty (std):", std_p[0])

# ------------------------------------------------------------------------------
# 10. Calibration Curve
# ------------------------------------------------------------------------------
all_probs, all_labels = [], []
with torch.no_grad():
    for xb, yb_cls, _ in val_loader:
        xb = xb.to(device)
        logp, _ = model(xb)[:2]
        calibrated_logp = temp_scaler(logp.cpu())
        probs = torch.exp(calibrated_logp).numpy()
        all_probs.extend(probs.max(axis=1))
        all_labels.extend((probs.argmax(axis=1) == yb_cls.numpy()).astype(int))

prob_true, prob_pred = calibration_curve(all_labels, all_probs, n_bins=10)
plt.figure()
plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0,1],[0,1],'k--')
plt.xlabel("Predicted Confidence")
plt.ylabel("Empirical Accuracy")
plt.title("Calibration Curve")
plt.show()

# ------------------------------------------------------------------------------
# 11. Decision Boundaries (2D slices)
# ------------------------------------------------------------------------------
model.eval()
grid_size = 200
x_vals = np.linspace(-3, 3, grid_size)
y_vals = np.linspace(-3, 3, grid_size)
fig, axes = plt.subplots(1, 3, figsize=(15,4))

for (i,j), ax in zip([(0,1),(2,3),(0,2)], axes):
    xx, yy = np.meshgrid(x_vals, y_vals)
    grid = np.zeros((grid_size*grid_size, 4), dtype=np.float32)
    grid[:, i] = xx.ravel()
    grid[:, j] = yy.ravel()
    xb_grid = torch.from_numpy(grid)
    with torch.no_grad():
        logp, _ , _ = model(xb_grid.to(device))
        cls_pred = logp.exp().argmax(dim=1).cpu().numpy()
    Z = cls_pred.reshape(grid_size, grid_size)
    ax.contourf(xx, yy, Z, levels=[-0.5,0.5,1.5,2.5], alpha=0.6, cmap='Accent')
    ax.scatter(
        dataset.X[:, i].numpy(),
        dataset.X[:, j].numpy(),
        c=dataset.y_cls.numpy(),
        s=5,
        cmap='Accent',
        edgecolor='k',
        linewidth=0.2
    )
    ax.set_xlabel(f"Feature {i}")
    ax.set_ylabel(f"Feature {j}")
    ax.set_title(f"Boundary ({i} vs {j})")
plt.tight_layout()
plt.show()

# ------------------------------------------------------------------------------
# 12. t-SNE of Hidden Representations
# ------------------------------------------------------------------------------
features, labels = [], []
with torch.no_grad():
    for xb, yb_cls, _ in val_loader:
        xb = xb.to(device)
        _, _, h = model(xb)
        features.append(h.cpu().numpy())
        labels.append(yb_cls.numpy())
features = np.concatenate(features)
labels   = np.concatenate(labels)

tsne = TSNE(n_components=2, perplexity=30, random_state=42)
emb  = tsne.fit_transform(features)

plt.figure(figsize=(6,6))
for cls in np.unique(labels):
    idx = labels == cls
    plt.scatter(emb[idx,0], emb[idx,1], s=5, label=f"Class {cls}")
plt.legend()
plt.title("t-SNE of Hidden Representations")
plt.show()